# Creating the dataset for Movie Posters genre classification


**Unzipping the archive.zip**


In [1]:
import zipfile


In [2]:
with zipfile.ZipFile('C:\\Users\\krzys\\Desktop\\Thesis\\code\\archive.zip', 'r') as zip_ref:
    zip_ref.extractall('C:\\Users\\krzys\\Desktop\\Thesis\\code\\archive')


**Accessing the CSV file to retrieve file name and genres**

In [3]:
import csv


In [4]:
# moving posters to the destination file with label
import shutil
import os


def progress_bar(current, total, bar_length=20):
    fraction = current / total

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)


def copy_to_dir(src: str, dst: str):
    dst_folder = dst.split('\\')[:-1]
    dst_path = ''
    for word in dst_folder:
        dst_path += word + '\\'
    dst_path = dst_path[:-1]

    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    shutil.copyfile(src, dst)


In [5]:
import sys
from PIL import Image


source_dir = r'C:\Users\krzys\Desktop\Thesis\code\archive\SampleMoviePosters\\'
dest_dir = r'C:\Users\krzys\Desktop\Thesis\code\movie_posters_dataset\\'

with open(r'C:\Users\krzys\Desktop\Thesis\code\archive\MovieGenre.csv', encoding='cp850') as csv_file:
    csv_reader = csv.reader(csv_file)
    line_count = 0
    corrupted_count = 0
    no_genre_count = 0
    for row in csv_reader:  # 40109 rows
        line_count += 1

        if line_count == 0:  # skipping header
            line_count += 1
        else:

            if row[4] == '':  # removing images with no genre assigned
                no_genre_count = no_genre_count + 1
                continue

            try:
                # check for corrupted files
                img = Image.open(source_dir + row[0] + '.jpg')
                img.verify()
                img.close()
                genres = row[4].split('|')  # str of genres to the list

                for genre in genres:  # for creation of separate folders of label\poster.jpg
                    copy_to_dir(
                        source_dir + row[0] + '.jpg', dest_dir + genre + '\\' + row[0] + '.jpg')

            except (IOError, SyntaxError) as e:
                # print('Corrupted file: ' + row[0] + ' ' + row[2])
                corrupted_count = corrupted_count + 1

        progress_bar(line_count, 40109)
    print(f'Processed {line_count} lines.\nFound {corrupted_count} corrupted images.\nFound {no_genre_count} images without assigned genre.\nIn the end {line_count - corrupted_count - no_genre_count} correct images.')


Progress: [------------------->] 100%
Processed 40109 lines.
Found 824 corrupted images.
Found 145 images without assigned genre.
In the end 39140 correct images.


**Creating .txt file with labels**

In [6]:
folder = r'C:\\Users\\krzys\\Desktop\\Thesis\\code\\movie_posters_dataset'

sub_folders = [name for name in os.listdir(
    folder) if os.path.isdir(os.path.join(folder, name))]


In [7]:
file = open('train.txt', 'w')
DIR = r'movie_posters_dataset\\'

for sf in sub_folders:
    for name in os.listdir(DIR + sf):
        file.writelines(sf + '/' + name + '\n')

file.close()
